In [110]:
import psycopg2
import pyspark

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [10]:
conn = psycopg2.connect(    
    host = "10.0.21.134", # ip de mon windows sur lequel est installé postgrés
    dbname = "Talend",
    user="postgres",
    password = "admin",
    port = 5432
)

In [13]:
cur = conn.cursor()

In [15]:
cur.execute("""select * from clients;""")

In [16]:
a = cur.fetchall()

In [17]:
print(a)

[(1, 'Louise', 'Marie', datetime.datetime(1986, 1, 28, 0, 0), 35324, 1, 5), (2, 'Arthur', 'Guehery', datetime.datetime(1987, 1, 27, 0, 0), 32631, 2, 2), (3, 'Amine', 'Hanouna', datetime.datetime(1989, 7, 11, 0, 0), 17695, 2, 4), (4, 'Manon', 'Lafouine', datetime.datetime(1960, 3, 22, 0, 0), 33376, 1, 5), (5, 'Gabriel', 'Cortez', datetime.datetime(1956, 10, 6, 0, 0), 10264, 2, 1), (6, 'Alice', 'Bartez', datetime.datetime(1982, 3, 14, 0, 0), 99814, 1, 5), (7, 'Lea', 'Marcel', datetime.datetime(1992, 10, 9, 0, 0), 13376, 1, 8), (8, 'Handy', 'Orton', datetime.datetime(1967, 11, 15, 0, 0), 73376, 2, 4), (9, 'Kakou', 'Bernadette', datetime.datetime(1965, 8, 9, 0, 0), 53376, 1, 2), (10, 'Bea', 'Martine', datetime.datetime(1983, 10, 10, 0, 0), 35000, 1, 9), (11, 'Maurice', 'Greene', datetime.datetime(1984, 2, 8, 0, 0), 72000, 1, 10)]


In [21]:
conn.commit()

In [22]:
cur = conn.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS florian2 (Id INT, Name VARCHAR(255));""")
conn.commit()  # Assurez-vous de commit les changements

In [23]:
cur.execute("""
            INSERT INTO florian2(Id, Name) VALUES
            (1,'Mike'),
            (2, 'Lise')            
            """)

In [24]:
conn.commit()

# exercice brief stephane

In [26]:
spark = SparkSession.builder.master("local[*]").appName("MigrationExample").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/09/11 11:40:54 WARN Utils: Your hostname, GIGABYTENEJ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/11 11:40:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 11:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### creation du dataframe

In [27]:
matrix = [
[' Ricky', 4879320143, '2022-09-05 11:23:59', True],
['Simon', 4879320143, '2022-09-05 11:23:59', True],
['Sergio', 4879320143, '2022-09-05 11:23:59', True],
['Pedrito', 4879320143, '2022-09-07 11:23:59', True],
['jJuenito', 4879320143, '2022-09-08 11:23:59', True],
['Pepe', 4879320143, '2022-09-09 11:23:59', True],
['Liusa', 4879320143, '2022-09-05 11:23:59', True],
]

columns = ['name', 'dni', 'date', 'is_human']

In [28]:
spark_df = spark.createDataFrame(matrix, columns)

### ajout colonne constant et ajouter des données avec lit

In [29]:
df_with_constant = spark_df.withColumn( "constant\_value", F.lit(1))
df_with_constant.show()

+--------+----------+-------------------+--------+---------------+
|    name|       dni|               date|is_human|constant\_value|
+--------+----------+-------------------+--------+---------------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|              1|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|              1|
+--------+----------+-------------------+--------+---------------+



In [36]:
df_with_constant.printSchema()

root
 |-- name: string (nullable = true)
 |-- dni: long (nullable = true)
 |-- date: string (nullable = true)
 |-- is_human: boolean (nullable = true)
 |-- constant\_value: integer (nullable = false)



### creation colonne avec condition

In [41]:
df = df_with_constant.withColumn("status", F.when(df_with_constant.date >= "32022-09-070", "old").otherwise("young"))
df.show()

+--------+----------+-------------------+--------+---------------+------+
|    name|       dni|               date|is_human|constant\_value|status|
+--------+----------+-------------------+--------+---------------+------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1| young|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1| young|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1| young|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1| young|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1| young|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|              1| young|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|              1| young|
+--------+----------+-------------------+--------+---------------+------+



### concatenation de 2 colonnes

In [42]:
df2 = df.select(F.concat(df.dni, df.status, df.is_human).alias("arr"))
df2.show()

+-------------------+
|                arr|
+-------------------+
|4879320143youngtrue|
|4879320143youngtrue|
|4879320143youngtrue|
|4879320143youngtrue|
|4879320143youngtrue|
|4879320143youngtrue|
|4879320143youngtrue|
+-------------------+



### month, year, quarter: Extraire le mois, l'année et le trimestre

In [53]:
Df = df_with_constant.withColumn('years',F.year(F.to_timestamp('date','yyyy-MM-dd hh:mm:ss')))\
    .withColumn('months',F.month(F.to_timestamp('date','yyyy-MM-dd hh:mm:ss')))\
    .withColumn('quarter',F.quarter(F.to_timestamp('date','yyyy-MM-dd hh:mm:ss')))
Df.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+
|    name|       dni|               date|is_human|constant\_value|years|months|quarter|
+--------+----------+-------------------+--------+---------------+-----+------+-------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1| 2022|     9|      3|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1| 2022|     9|      3|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|              1| 2022|     9|      3|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|
+--------+----------+-------------------+--------+---------------+-----+------+-------+



### Abs et floor

In [62]:
Df = Df.withColumn( "-constant\_value", F.lit(-1.5))
Df.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+
|    name|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1| 2022|     9|      3|            -1.5|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1| 2022|     9|      3|            -1.5|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|              1| 2022|     9|      3|            -1.5|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|     

In [63]:
DfAbs = Df.withColumn("AbsConst", F.abs("-constant\_value"))
DfAbs.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+
|    name|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|              1| 2022|     9|      3| 

In [65]:
DfFloor = DfAbs.withColumn("Floor",F.floor("AbsConst"))
DfFloor.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+
|    name|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|
|    Pepe|4879320143|2022-09-09 11:23:59

### concat_ws

In [66]:
DfConcatWs = DfFloor.withColumn("humain-constant", F.concat_ws('-', DfFloor.is_human, DfFloor.AbsConst))
DfConcatWs.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+
|    name|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|
|jJuenito|4879320143|2022-09-08 11:23:59|    tru

### Array_distinct, collect_list, collect_set

In [68]:
df = spark.createDataFrame([([1, 2, 3, 2],), ([4, 5, 5, 4],)], ['data'])
df.select(F.array_distinct(df.data)).collect()

[Row(array_distinct(data)=[1, 2, 3]), Row(array_distinct(data)=[4, 5])]

### split et trim

In [76]:
splitDF = DfConcatWs.withColumn('maxime',F.split(DfConcatWs.date, ' ', 2))
splitDF.show(truncate = False)

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+----------------------+
|name    |dni       |date               |is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|maxime                |
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+----------------------+
| Ricky  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |[2022-09-05, 11:23:59]|
|Simon   |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |[2022-09-05, 11:23:59]|
|Sergio  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |[2022-09-05, 11:23:59]|
|Pedrito |4879320143|2022-09-07 11:23:59|true 

In [78]:
trimDF = DfConcatWs.withColumn('chems',F.trim(DfConcatWs.name))
trimDF.show(truncate = False)

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+
|name    |dni       |date               |is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|chems   |
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+
| Ricky  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Ricky   |
|Simon   |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Simon   |
|Sergio  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Sergio  |
|Pedrito |4879320143|2022-09-07 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5   

### explode

In [79]:
data = [("John", ["apple", "banana", "orange"]),
("Alice", ["grape", "berry"]),
("Bob", ["melon", "peach", "kiwi"])]

df = spark.createDataFrame(data, ["name", "fruits"])

In [81]:
df.show()

+-----+--------------------+
| name|              fruits|
+-----+--------------------+
| John|[apple, banana, o...|
|Alice|      [grape, berry]|
|  Bob|[melon, peach, kiwi]|
+-----+--------------------+



In [82]:
exploded_df = df.select(df.name, F.explode(df.fruits).alias("fruit"))
exploded_df.show()

+-----+------+
| name| fruit|
+-----+------+
| John| apple|
| John|banana|
| John|orange|
|Alice| grape|
|Alice| berry|
|  Bob| melon|
|  Bob| peach|
|  Bob|  kiwi|
+-----+------+



### upper et lower

In [83]:
upperDf = trimDF.withColumn('NAME',F.upper(trimDF.name))
upperDf.show(truncate = False)

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+
|NAME    |dni       |date               |is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|chems   |
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+
| RICKY  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Ricky   |
|SIMON   |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Simon   |
|SERGIO  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Sergio  |
|PEDRITO |4879320143|2022-09-07 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5   

In [87]:
lowerDf = upperDf.withColumn('lowerName',F.lower(upperDf.NAME))
lowerDf.show(truncate = False)

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+
|NAME    |dni       |date               |is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|chems   |lowerName|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+
| RICKY  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Ricky   | ricky   |
|SIMON   |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Simon   |simon    |
|SERGIO  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Sergio  |sergio   |
|PEDRITO |4879320143|2022-09-07 11:23:59|true    |1              |2022

### select 

In [94]:
upperDf.select("chems","name","is_human").show()

+--------+--------+--------+
|   chems|    name|is_human|
+--------+--------+--------+
|   Ricky|   RICKY|    true|
|   Simon|   SIMON|    true|
|  Sergio|  SERGIO|    true|
| Pedrito| PEDRITO|    true|
|jJuenito|JJUENITO|    true|
|    Pepe|    PEPE|    true|
|   Liusa|   LIUSA|    true|
+--------+--------+--------+



### withcolumn

In [97]:
withCols = lowerDf.withColumn("stephane", F.upper(lowerDf.lowerName))
withCols.show()


+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+
|    NAME|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Simon|    simon|   SIMON|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|  Sergio|   sergio|  SERGIO|
| PEDRITO|487932

### withcolumnsRenamed

In [98]:
rené = withCols.withColumnRenamed('dni','DNI')
rené.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Simon|    simon|   SIMON|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|  Sergio|   sergio|  SERGIO|
| PEDRITO|487932

### drop

In [99]:
renée = rené.drop("AbsConst")
renée.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+-----+---------------+--------+---------+--------+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|Floor|humain-constant|   chems|lowerName|stephane|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+-----+---------------+--------+---------+--------+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|    1|       true-1.5|   Simon|    simon|   SIMON|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|    1|       true-1.5|  Sergio|   sergio|  SERGIO|
| PEDRITO|4879320143|2022-09-07 11:23:59|    true|              1| 202

### join

In [100]:
rené.join(renée, rené.NAME == renée.NAME, "inner").show()

24/09/11 15:24:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+-----+---------------+--------+---------+--------+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|Floor|humain-constant|   chems|lowerName|stephane|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+-----+---------------+--------+---------+--------+
|   RICKY|4879320143|2022-09-05 11:23:59|    true

### groupby et agg

In [101]:
 # créer d'abord une colonne jour pour faire mon groupby dessus
 
renéDaté = rené.withColumn('jour',F.day(F.to_timestamp('date','yyyy-MM-dd hh:mm:ss')))
renéDaté.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+----+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|jour|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+----+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY|   5|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Simon|    simon|   SIMON|   5|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|  Sergio|   sergio| 

In [103]:
renéGroupé = renéDaté.groupBy(renéDaté.jour).agg({"Floor":"sum"})
renéGroupé.show()

+----+----------+
|jour|sum(Floor)|
+----+----------+
|   5|         4|
|   7|         1|
|   8|         1|
|   9|         1|
+----+----------+



### drop duplicates

In [104]:
deduplicates = rené.dropDuplicates()
deduplicates.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Simon|    simon|   SIMON|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|  Sergio|   sergio|  SERGIO|
| PEDRITO|487932

### where filter

In [105]:
filtreDf  = deduplicates.filter(deduplicates.DNI != 4879320143)
filtreDf.show()

+----+---+----+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-----+---------+--------+
|NAME|DNI|date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|chems|lowerName|stephane|
+----+---+----+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-----+---------+--------+
+----+---+----+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-----+---------+--------+



### when

In [106]:
df88 = rené.withColumn("STATUS", F.when(rené.date >= "32022-09-070", "old").otherwise("GAMIN"))
df88.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+------+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|STATUS|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+------+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY| GAMIN|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Simon|    simon|   SIMON| GAMIN|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|  Sergio| 

### astype

In [112]:
rené.printSchema()

root
 |-- NAME: string (nullable = true)
 |-- DNI: long (nullable = true)
 |-- date: string (nullable = true)
 |-- is_human: boolean (nullable = true)
 |-- constant\_value: integer (nullable = false)
 |-- years: integer (nullable = true)
 |-- months: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- -constant\_value: double (nullable = false)
 |-- AbsConst: double (nullable = false)
 |-- Floor: long (nullable = true)
 |-- humain-constant: string (nullable = false)
 |-- chems: string (nullable = true)
 |-- lowerName: string (nullable = true)
 |-- stephane: string (nullable = true)



In [116]:
intDni = rené.withColumn("DNII", rené.DNI.cast(IntegerType()))
intDni.printSchema()

root
 |-- NAME: string (nullable = true)
 |-- DNI: long (nullable = true)
 |-- date: string (nullable = true)
 |-- is_human: boolean (nullable = true)
 |-- constant\_value: integer (nullable = false)
 |-- years: integer (nullable = true)
 |-- months: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- -constant\_value: double (nullable = false)
 |-- AbsConst: double (nullable = false)
 |-- Floor: long (nullable = true)
 |-- humain-constant: string (nullable = false)
 |-- chems: string (nullable = true)
 |-- lowerName: string (nullable = true)
 |-- stephane: string (nullable = true)
 |-- DNII: integer (nullable = true)



### isin

In [122]:
renéné = rené.withColumn('Rname', rené.NAME.isin('PEDRITO','PEPE'))
renéné.show()

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+-----+
|    NAME|       DNI|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|   chems|lowerName|stephane|Rname|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+--------+-----+
|   RICKY|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Ricky|    ricky|   RICKY|false|
|   SIMON|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Simon|    simon|   SIMON|false|
|  SERGIO|4879320143|2022-09-05 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|  Sergio|   ser

### lower upper dans mon cas

In [123]:
erné = lowerDf.withColumn('lowerName',F.upper(lowerDf.is_human))
erné.show(truncate = False)

+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+
|NAME    |dni       |date               |is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|chems   |lowerName|
+--------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+--------+---------+
| RICKY  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Ricky   |TRUE     |
|SIMON   |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Simon   |TRUE     |
|SERGIO  |4879320143|2022-09-05 11:23:59|true    |1              |2022 |9     |3      |-1.5            |1.5     |1    |true-1.5       |Sergio  |TRUE     |
|PEDRITO |4879320143|2022-09-07 11:23:59|true    |1              |2022

### alias

In [130]:
erné.select(erné.is_human.alias('is_human2')).collect()

[Row(is_human2=True),
 Row(is_human2=True),
 Row(is_human2=True),
 Row(is_human2=True),
 Row(is_human2=True),
 Row(is_human2=True),
 Row(is_human2=True)]

### isnull

In [131]:
erné.filter(erné.is_human.isNull()).collect()

[]

### like et rlike

In [132]:
# like
likeP = erné.filter(erné.NAME.like('P%'))
likeP.show()

+-------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-------+---------+
|   NAME|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|  chems|lowerName|
+-------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-------+---------+
|PEDRITO|4879320143|2022-09-07 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|Pedrito|     TRUE|
|   PEPE|4879320143|2022-09-09 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5|   Pepe|     TRUE|
+-------+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-------+---------+



In [134]:
# rlike
erné.filter(F.col("NAME")
    .rlike("(?i)^*PE$")
  ).show()

+----+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-----+---------+
|NAME|       dni|               date|is_human|constant\_value|years|months|quarter|-constant\_value|AbsConst|Floor|humain-constant|chems|lowerName|
+----+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-----+---------+
|PEPE|4879320143|2022-09-09 11:23:59|    true|              1| 2022|     9|      3|            -1.5|     1.5|    1|       true-1.5| Pepe|     TRUE|
+----+----------+-------------------+--------+---------------+-----+------+-------+----------------+--------+-----+---------------+-----+---------+



### regexp_replace

In [ ]:
new_df = erné.withColumn("New_Comments", F.regexp_replace("NAME", "SIMON", "MONSI"))
new_df.show()

### str.contains

In [ ]:
new_filtered_df = new_df.filter(F.col("NAME").contains("RICKY"))
new_filtered_df.show()

### différence entre collect et show

_ collect : recupère toutes les données du jeu et les renvoie au driver (utiles sur des petites jeux de données, mais déconseillé pour les gros jeux car risque de surcharge de la mémoire) \
_ show permet d'afficher un nombre de lignes limité sur la console (IDE terminal) elle ne renvoie pas les données au driver => moins gourmand en resosurces

### différence entre RDD et count()

_ RDD structude de données de base de spark => collection d'éléments pouvant être traités en parrallèle \
_ count est une action qui permet d'obtenir le nombre d'élément contenu dans un ensemble de données